In [3]:
from transformers import  utils, Trainer, TrainingArguments, ElectraTokenizer, ElectraForSequenceClassification
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import numpy as np
from datasets import Dataset, DatasetDict
import torch
from accelerate import Accelerator
import os
from bertviz import model_view
utils.logging.set_verbosity_error()

In [4]:
arguments = pd.read_csv('../data/fallacyRecognition/arguments-en-2018-01-15.tsv', sep='\t')

In [7]:
arguments = arguments[['Author', 'Topic', 'Stance', 'Intended Fallacy', 'Voted Fallacy', 'Number of Votes', 'Text']]

In [14]:
arguments.loc[arguments['Intended Fallacy']=='Red Herring', 'Intended Fallacy'] = 'fallacy of relevance'

,Author,Topic,Stance,Intended Fallacy,Voted Fallacy,Number of Votes,Text
0,bbc9da056b6a9234,Should cellphones be used during class?,contra,Appeal to Emotion,-,0.0,"No, imagine you were a teacher and everybody w..."
1,a4f93c2c8cc92a8c,Has anyone been on the moon?,contra,fallacy of relevance,-,1.0,"The moon is so far away, we should focus on ou..."
2,3f4339e14df7d077,Is it justified to develop nuclear energy for ...,contra,Irrelevant Authority,Irrelevant Authority,6.0,"The green party in Germany has the opinion, th..."
3,e01c557ce0fdb585,Do social networking sites have a positive imp...,contra,fallacy of relevance,-,0.0,"No, not at all. Nowadays kids spend all time p..."
4,d038663d70f77d9b,Should driving under the influence of alcohol ...,pro,No Fallacy,No Fallacy,8.0,"Yes, whoever drinks and drives has a bad reac..."
...,...,...,...,...,...,...,...
1339,e01c557ce0fdb585,Is there a good reason for the American war on...,contra,Appeal to Emotion,-,2.0,"Sure, let's make America great again! Go ask o..."
1340,cc1eee636691c425,The greek exit from the EU is a necessary step.,contra,No Fallacy,-,2.0,"No. Greece is one of the countries, that, in i..."
1341,58acb7acccce58ff,Are single sex schools more effective than co-...,contra,No Fallacy,-,0.0,Interestingly the boys I know from single sex ...
1342,4ccadeb24a63a251,Has television destroyed the communication amo...,pro,No Fallacy,No Fallacy,3.0,"It has, earlier the people talked more with ea..."
